# 0 step

In [1]:
!pip install transformers
!pip install sentencepiece

## installs

In [2]:
pip install -U pandas-profiling[notebook]

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install openpyxl

In [4]:
!pip install xgboost

In [5]:
!pip install ipywidgets

In [6]:
!pip install wandb

In [7]:
!pip install python-dotenv

In [8]:
import os
os.chdir("/home/jupyter/makethon/TUM-Makeathon-22")

In [9]:
!pip install torchvision

## imports

In [10]:
import pandas as pd
import numpy as np
import scipy
from research import tools
from importlib import reload

# visulization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm


# Basic
import json
import re
import os
import random


# Machine learning
import sklearn
from sklearn.model_selection import (
    train_test_split, GridSearchCV,
    cross_val_score, KFold
)
os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Deep Learning
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils import checkpoint
from torch.utils.data import DataLoader, Dataset

# Hugging face
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup



# XGB
import xgboost as xgb
os.environ['WANDB_SILENT']="true"

In [11]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
font = {'family':'Helvetica, Ariel',
        'weight':'normal',
        'size':12}
plt.rc('font', **font)
sns.set(rc={"figure.dpi": 300, 'savefig.dpi': 300})
sns.set_context('notebook')
sns.set_style("ticks")
FIG_FONT = dict(family="Helvetica, Ariel", weight="bold", color="#7f7f7f")
sns.set_palette('Spectral')

In [12]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [13]:
class CFG:
    wandb=False
    competition='makethon'
    debug=False
    apex=False # automatic mixed precision
    print_freq=10
    num_workers=4
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5 # The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0 following a half-cosine).
    num_warmup_steps=0
    epochs=4
    encoder_lr=2e-3
    decoder_lr=2e-3
    min_lr=1e-3
    eps=1e-6
    betas=(0.9, 0.999)
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['risk']  # TODO
    seed=42
    train=True

if CFG.wandb:
    from dotenv import load_dotenv

    load_dotenv('research/settings.env')
    wandb_api = os.environ['WANDB']
    run = wandb.init(
        project='AI4CODE', 
        name=CFG.model_name,
        config=class2dict(CFG),
        # group=CFG.model_name,
        job_type="train"
    )

In [14]:
seed_everything(seed=42)
LOGGER = tools.get_logger(filename='logs')

# Data Import

In [15]:
import warnings
warnings.filterwarnings("ignore")
import math
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm
from transformers import AutoTokenizer
import time
import os
import gc
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
# from pandas_profiling import ProfileReport
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.utils import class_weight
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
df = pd.read_excel(
    "research/data/appliedAI_Use Case Library - Risk Class_TUM.ai.xlsx"
)
df['risk'] = (
    df['Is the AI System high-risk or low risk?'].replace(
        {
            'low-risk' : 0,
            'high-risk': 1,
            'It is unclear': None
        }
    )
)
df = df.dropna(subset=['risk'])

In [25]:
df['human']=(
    df['Does the AI System interact with humans?'].replace(
        {
            'no' : 0,
            
            'yes': 1,
            'yes - real time?': 1,
            'real time?': 1,
        }
    )
)
df = df.dropna(subset=['human'])

In [26]:
train_df = df[['Description', 'Business Challenge', 'AI System', 'human']]
train_df = train_df.reset_index()

# Dataset

In [493]:
class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.description = df['description'].values
        # self.biz_challendge = df['Business Challenge'].values
        # self.ai_system = df['AI System'].values
        self.labels = nn.functional.one_hot(
            torch.LongTensor(labelencoder.transform(df['tags'])),
            num_classes=len(CATEGORIES)
        )
        # self.labels = df['label'].values
        self.tokenizer = AutoTokenizer.from_pretrained(
            "bert-base-uncased",
            do_lower_case=True
        )

    def __len__(self):
        return len(self.description)
    
    def tokenize(self, data):
        return self.tokenizer.encode_plus(
            data,
            None,
            add_special_tokens=False,
            max_length=CFG.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )

    def __getitem__(self, item):
        inputs = self.tokenize(str(self.description[item]))
        labels = torch.tensor(self.labels[item], dtype=torch.float)
        ids = torch.LongTensor(inputs['input_ids'])
        mask = torch.LongTensor(inputs['attention_mask'])
        return ids, mask, labels

# Model

In [494]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.distill_bert = AutoModel.from_pretrained(
            "bert-base-uncased"
        )
        
        self.top1 = nn.Linear(768*1, 768)
        self.top2 = nn.Linear(768, 768)
        self.top3 = nn.Linear(768, len(CATEGORIES))  # classify the data into N laws described in the training dataset

        self.dropout1 = torch.nn.Dropout(p=0.3)
        self.dropout2 = torch.nn.Dropout(p=0.3)
        self.sigm = nn.Sigmoid()

    def forward(self, ids, mask):
        x = self.distill_bert(ids, mask)[0][:, 0, :]
        # select the first token only as it is trained for classifier
        x = F.relu(self.top1(x))
        x = self.dropout1(x)
        
        x = F.relu(self.top2(x))
        x = self.dropout2(x)
        
        x = self.top3(x)
        return F.softmax(x)

In [495]:
def get_optimizer(net):
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, net.parameters()), lr=3e-4, betas=(0.9, 0.999),
        eps=1e-08)
    return optimizer

def adjust_lr(optimizer, epoch):
    if epoch < 1:
        lr = 5e-5
    elif epoch < 2:
        lr = 4e-5
    elif epoch < 5:
        lr = 3e-5
    else:
        lr = 2e-5

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr
def read_data(data):
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()

In [496]:
def validate(model, val_loader):
    model.eval()
    tbar = tqdm(val_loader)
    preds = []
    labels = []

    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            pred = model(*inputs)

            preds.append(pred.detach().cpu().numpy())
            labels.append(target.detach().cpu().numpy())
    return np.concatenate(labels), np.concatenate(preds)

In [497]:
def train_fn(model, train_loader, val_loader, criterion, weight_,  epochs):
    np.random.seed(CFG.seed)
    optimizer = get_optimizer(model)
    for e in range(epochs):   
        model.train()
        tbar = tqdm(train_loader)
        lr = adjust_lr(optimizer, e)
        loss_list = []
        preds = []
        labels = []
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)
            optimizer.zero_grad()
            pred = model(*inputs)

            loss = criterion(pred, target)
            loss = loss.mean()

            loss.backward()
            optimizer.step()
            
            loss_list.append(loss.detach().cpu().item())
            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())
            
            avg_loss = np.round(np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")
        print(f"MEAN TRAIN LOSS: {np.mean(loss_list)}")
        # print(pred)
        # print(target)
    y_val, y_pred = validate(model, val_loader)
    val_loss = mean_absolute_error(y_val, y_pred)
    print(f"VAL LOSS: {val_loss}")
    return model, y_pred, y_val

In [498]:
train = pd.concat([ideas_from_law_df, train_df[train_df['risk']==0]], axis=0)

In [499]:
torch.cuda.empty_cache()
gc.collect()
skf = StratifiedKFold(n_splits=8, shuffle=True)

for train_index, test_index in skf.split(train.values, train['label']):
    model = CustomModel()
    model.to(device)
    print("Build the model")
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    
    train_dataset = TrainDataset(CFG, X_train)
    valid_dataset = TrainDataset(CFG, X_test)
    criterion = nn.BCEWithLogitsLoss()
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        # num_workers=-1,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=10,
        shuffle=False,
        # num_workers=-1,
        pin_memory=True,
        drop_last=False,
    )
    
    model, y_pred, y_val = train_fn(model, train_loader, valid_loader, criterion, None, epochs=4)
    print("TRAIN DATA VALIDATION F1: ", f1_score(np.argmax(y_pred, axis=0),
        np.argmax(y_val, axis=0), average='micro')
         )
    
    valid_dataset = TrainDataset(CFG, test_filtered)
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=10,
        shuffle=False,
        # num_workers=-1,
        pin_memory=True,
        drop_last=False,
    )
    y_val, y_pred = validate(model, valid_loader)
    val_loss = mean_absolute_error(y_val, y_pred)
    print(f"ORIGINAL VAL VALIDATION: {val_loss}")
    print("ORIGINAL DATASET VALIDATION F1: ", f1_score(np.argmax(y_pred, axis=0),
        np.argmax(y_val, axis=0), average='micro')
         )
    print("-"*40)
    
    torch.cuda.empty_cache()
    gc.collect()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Build the model


  0%|          | 0/45 [00:00<?, ?it/s]

MEAN TRAIN LOSS: 0.7111934039327833


  0%|          | 0/45 [00:00<?, ?it/s]

...

MEAN TRAIN LOSS: 0.7080967995855544


  0%|          | 0/11 [00:00<?, ?it/s]

VAL LOSS: 0.06702466309070587
TRAIN DATA VALIDATION F1:  0.24


  0%|          | 0/2 [00:00<?, ?it/s]

ORIGINAL VAL VALIDATION: 0.07999438792467117
ORIGINAL DATASET VALIDATION F1:  0.04
----------------------------------------


In [ ]:
y_pred[:3]

# y_val[:3]

In [ ]:
test_dataset = TrainDataset(CFG, X_test)
valid_loader = DataLoader(
        valid_dataset,
        batch_size=13,
        shuffle=False,
        pin_memory=True,
        drop_last=False,
    )

In [ ]:
preds = []
labels = []
for idx, data in valid_loader:
    inputs, target = read_data(data)
    pred = model(*inputs)
    preds.append(pred.detach().cpu().numpy())
    labels.append(target.detach().cpu().numpy())
np.concatenate(labels), np.concatenate(preds)

In [685]:
winners = []
for row in y_pred:
    winners.extend(np.argsort(row)[:3])

In [688]:
import collections

In [689]:
collections.Counter(winners)

Counter({17: 34,
         3: 37,
         11: 37,
         13: 55,
         18: 9,
         1: 19,
         6: 13,
         9: 1,
         15: 1,
         7: 1,
         23: 1,
         16: 3,
         19: 1,
         8: 1})

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, y_pred)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, y_pred)
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Logistic')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()
cm = confusion_matrix(y_val, [1 if x>0.25 else 0 for x in y_pred])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()